## 30s k-fold

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from toolkit import plot_confusion_matrix
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from toolkit import to_3_classes
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainL = to_3_classes.to_3_classes(trainL)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 3)

print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 3)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=True)


In [ ]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(learning_rate=0):
    model = Sequential()
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(3, activation = "softmax"))
    
    adam = Adam(lr = learning_rate)
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
    
    return model


batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]
parameter_grid = dict(batch_size=batch_S, learning_rate=learning_rate)
model = KerasClassifier(build_fn = create_model, epochs=80, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, cv=kf, verbose = 1)
grid_result = grid.fit(trainData, trainLabel)

print('Finish training!')

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/80
8120/8120 [==============================] - 13s 2ms/step - loss: 0.9208 - acc: 0.5813
Epoch 2/80
8120/8120 [==============================] - 8s 926us/step - loss: 0.8990 - acc: 0.5865
Epoch 3/80
8120/8120 [==============================] - 8s 978us/step - loss: 0.8674 - acc: 0.5905
Epoch 4/80
8120/8120 [==============================] - 8s 997us/step - loss: 0.7873 - acc: 0.6426
Epoch 5/80
8120/8120 [==============================] - 8s 990us/step - loss: 0.7366 - acc: 0.6666
Epoch 6/80
8120/8120 [==============================] - 8s 988us/step - loss: 0.7125 - acc: 0.6791
Epoch 7/80
8120/8120 [==============================] - 8s 992us/step - loss: 0.6995 - acc: 0.6788
Epoch 8/80
8120/8120 [==============================] - 8s 993us/step - loss: 0.6816 - acc: 0.6945
Epoch 9/80
8120/8120 [==============================] - 8s 994us/step - loss: 0.6669 - acc: 0.6935
Epoch 10/80
8120/8120 [==============================] - 8s 997us/step - loss: 0.6384 - acc: 0.7097
Epoch 11/8

8121/8121 [==============================] - 10s 1ms/step - loss: 0.9171 - acc: 0.5807
Epoch 2/80
8121/8121 [==============================] - 8s 976us/step - loss: 0.8944 - acc: 0.5871
Epoch 3/80
8121/8121 [==============================] - 8s 988us/step - loss: 0.8716 - acc: 0.5882
Epoch 4/80
8121/8121 [==============================] - 8s 991us/step - loss: 0.8056 - acc: 0.6284
Epoch 5/80
8121/8121 [==============================] - 8s 973us/step - loss: 0.7551 - acc: 0.6610
Epoch 6/80
8121/8121 [==============================] - 8s 992us/step - loss: 0.7215 - acc: 0.6792
Epoch 7/80
8121/8121 [==============================] - 8s 988us/step - loss: 0.6995 - acc: 0.6913
Epoch 8/80
8121/8121 [==============================] - 8s 978us/step - loss: 0.6781 - acc: 0.6942
Epoch 9/80
8121/8121 [==============================] - 8s 987us/step - loss: 0.6559 - acc: 0.7110
Epoch 10/80
8121/8121 [==============================] - 8s 978us/step - loss: 0.6388 - acc: 0.7178
Epoch 11/80
8121/8121

Epoch 2/80
8121/8121 [==============================] - 8s 944us/step - loss: 0.8922 - acc: 0.5925
Epoch 3/80
8121/8121 [==============================] - 8s 951us/step - loss: 0.8637 - acc: 0.5951
Epoch 4/80
8121/8121 [==============================] - 8s 960us/step - loss: 0.7971 - acc: 0.6385
Epoch 5/80
8121/8121 [==============================] - 8s 960us/step - loss: 0.7504 - acc: 0.6707
Epoch 6/80
8121/8121 [==============================] - 8s 958us/step - loss: 0.7086 - acc: 0.6798
Epoch 7/80
8121/8121 [==============================] - 8s 966us/step - loss: 0.6876 - acc: 0.6941
Epoch 8/80
8121/8121 [==============================] - 8s 959us/step - loss: 0.6694 - acc: 0.6972
Epoch 9/80
8121/8121 [==============================] - 8s 969us/step - loss: 0.6404 - acc: 0.7186
Epoch 10/80
8121/8121 [==============================] - 8s 943us/step - loss: 0.6307 - acc: 0.7168
Epoch 11/80
8121/8121 [==============================] - 8s 972us/step - loss: 0.6110 - acc: 0.7289
Epoch 12

...

8121/8121 [==============================] - 7s 869us/step - loss: 0.4129 - acc: 0.8339
Epoch 55/80
8121/8121 [==============================] - 7s 904us/step - loss: 0.4420 - acc: 0.8221
Epoch 56/80
8121/8121 [==============================] - 7s 856us/step - loss: 0.4054 - acc: 0.8398
Epoch 57/80
8121/8121 [==============================] - 7s 901us/step - loss: 0.4229 - acc: 0.8350
Epoch 58/80
8121/8121 [==============================] - 7s 869us/step - loss: 0.4084 - acc: 0.8409
Epoch 59/80
8121/8121 [==============================] - 7s 865us/step - loss: 0.3997 - acc: 0.8439
Epoch 60/80
8121/8121 [==============================] - 7s 868us/step - loss: 0.4055 - acc: 0.8382
Epoch 61/80
8121/8121 [==============================] - 7s 903us/step - loss: 0.4167 - acc: 0.8354
Epoch 62/80
8121/8121 [==============================] - 7s 869us/step - loss: 0.4086 - acc: 0.8393
Epoch 63/80
8121/8121 [==============================] - 7s 869us/step - loss: 0.3895 - acc: 0.8474
Epoch 64/80


Epoch 55/80
8121/8121 [==============================] - 8s 933us/step - loss: 0.4087 - acc: 0.8387
Epoch 56/80
8121/8121 [==============================] - 8s 932us/step - loss: 0.4113 - acc: 0.8398
Epoch 57/80
8121/8121 [==============================] - 7s 923us/step - loss: 0.4013 - acc: 0.8408
Epoch 58/80
8121/8121 [==============================] - 7s 909us/step - loss: 0.3976 - acc: 0.8388
Epoch 59/80
8121/8121 [==============================] - 7s 918us/step - loss: 0.3917 - acc: 0.8421
Epoch 60/80
8121/8121 [==============================] - 8s 925us/step - loss: 0.3909 - acc: 0.8482
Epoch 61/80
8121/8121 [==============================] - 7s 883us/step - loss: 0.3941 - acc: 0.8467
Epoch 62/80
8121/8121 [==============================] - 7s 919us/step - loss: 0.3900 - acc: 0.8458
Epoch 63/80
8121/8121 [==============================] - 7s 886us/step - loss: 0.3867 - acc: 0.8504
Epoch 64/80
8121/8121 [==============================] - 7s 877us/step - loss: 0.3975 - acc: 0.8474


8121/8121 [==============================] - 7s 892us/step - loss: 0.4571 - acc: 0.8154
Epoch 56/80
8121/8121 [==============================] - 7s 883us/step - loss: 0.4147 - acc: 0.8323
Epoch 57/80
8121/8121 [==============================] - 7s 920us/step - loss: 0.4111 - acc: 0.8381
Epoch 58/80
8121/8121 [==============================] - 7s 921us/step - loss: 0.4080 - acc: 0.8383
Epoch 59/80
8121/8121 [==============================] - 7s 911us/step - loss: 0.4119 - acc: 0.8351
Epoch 60/80
8121/8121 [==============================] - 7s 916us/step - loss: 0.4042 - acc: 0.8372
Epoch 61/80
8121/8121 [==============================] - 7s 920us/step - loss: 0.4011 - acc: 0.8413
Epoch 62/80
8121/8121 [==============================] - 8s 927us/step - loss: 0.3939 - acc: 0.8383
Epoch 63/80
8121/8121 [==============================] - 8s 939us/step - loss: 0.4086 - acc: 0.8350
Epoch 64/80
8121/8121 [==============================] - 7s 913us/step - loss: 0.3841 - acc: 0.8493
Epoch 65/80


8120/8120 [==============================] - 8s 959us/step - loss: 0.3004 - acc: 0.8852
Epoch 56/80
8120/8120 [==============================] - 8s 937us/step - loss: 0.2953 - acc: 0.8862
Epoch 57/80
8120/8120 [==============================] - 8s 936us/step - loss: 0.2989 - acc: 0.8851
Epoch 58/80
8120/8120 [==============================] - 7s 920us/step - loss: 0.2936 - acc: 0.8834
Epoch 59/80
8120/8120 [==============================] - 8s 928us/step - loss: 0.2891 - acc: 0.8922
Epoch 60/80
8120/8120 [==============================] - 7s 923us/step - loss: 0.2840 - acc: 0.8927
Epoch 61/80
8120/8120 [==============================] - 8s 928us/step - loss: 0.2747 - acc: 0.9005
Epoch 62/80
8120/8120 [==============================] - 7s 923us/step - loss: 0.2821 - acc: 0.8920
Epoch 63/80
8120/8120 [==============================] - 8s 924us/step - loss: 0.2847 - acc: 0.8917
Epoch 64/80
8120/8120 [==============================] - 8s 924us/step - loss: 0.2716 - acc: 0.8980
Epoch 65/80


Epoch 56/80
8121/8121 [==============================] - 7s 922us/step - loss: 0.3255 - acc: 0.8774
Epoch 57/80
8121/8121 [==============================] - 8s 928us/step - loss: 0.3223 - acc: 0.8797
Epoch 58/80
8121/8121 [==============================] - 8s 929us/step - loss: 0.3173 - acc: 0.8797
Epoch 59/80
8121/8121 [==============================] - 8s 929us/step - loss: 0.3134 - acc: 0.8838
Epoch 60/80
8121/8121 [==============================] - 7s 895us/step - loss: 0.3018 - acc: 0.8859
Epoch 61/80
8121/8121 [==============================] - 7s 905us/step - loss: 0.3518 - acc: 0.8718
Epoch 62/80
8121/8121 [==============================] - 7s 899us/step - loss: 0.3263 - acc: 0.8762
Epoch 63/80
8121/8121 [==============================] - 7s 919us/step - loss: 0.3177 - acc: 0.8780
Epoch 64/80
8121/8121 [==============================] - 8s 935us/step - loss: 0.3037 - acc: 0.8863
Epoch 65/80
8121/8121 [==============================] - 7s 921us/step - loss: 0.2954 - acc: 0.8893


In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean:%f (STD:%f) with: %r" % (mean, stdev, param))

In [ ]:
grid_result.best_score_

In [ ]:
grid_result.best_params_